In [1]:
import cv2
import matplotlib.pyplot as plt
import json
import sys

In [2]:
with open('/home/lab/cocodataset/annotations/annotations/person_keypoints_train2017.json') as json_file:
    json_data=json.load(json_file)

In [3]:
len(json_data['annotations'])/40000

6.561625

In [4]:
import math
math.ceil(len(json_data['annotations'])/40000)

7

# Lack of Ram  

## annotaions list split 40000  

## Num_keypoint > 13

In [5]:
list_json=[]
for i in range(math.ceil(len(json_data['annotations'])/40000)):
    if i !=6:
        list_json.append(json_data['annotations'][i*40000:(i+1)*40000])
    else:
        list_json.append(json_data['annotations'][i*40000:])

In [6]:
len(list_json)

7

In [7]:
len(list_json[0])

40000

In [8]:
def make_keypoint(keypoint,bbox):
    list_kp=[]
    if int(len(keypoint)/3) != len(keypoint)/3:
        print("NO Matching")
        sys.exit()
        
    for i in range(int(len(keypoint)/3)):
        coordinate=[]
        x=keypoint[i*3]
        y=keypoint[i*3+1]
        v=keypoint[i*3+2]
        coordinate.append(x)
        coordinate.append(y)
        coordinate.append(v)
        list_kp.append(coordinate)
    i_x=int(bbox[0])
    i_y=int(bbox[1])
    new_list=[]
    for i in list_kp:
        i[0]=i[0]-i_x
        i[1]=i[1]-i_y
        new_list.append(i)
    return new_list

In [9]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [10]:
def COCO_Make_dataset(json_data):
    img_path='/home/lab/cocodataset/train2017/train2017/000000'
    jpg='.jpg'
    list_img=[]
    kp_list=[]
    crowd=[]
    for i in tqdm(json_data):
        time.sleep(0.00000001)
        if i['num_keypoints']>13:
            bbox=i['bbox']
            img=i['image_id']
            keypoint=i['keypoints']
            crowd.append(i['iscrowd'])
            img=str(img)
            if len(img)>6:
                print("up 6-len name")
            while len(img)<6:
                img='0'+img
            image=cv2.imread(img_path+img+jpg)
            i_x=int(bbox[0])
            i_y=int(bbox[1])
            i_w=int(bbox[2])
            i_h=int(bbox[3])
            b_img=image[i_y : i_y+i_h , i_x : i_x+i_w]
            list_img.append(b_img)
            kp_list.append(make_keypoint(keypoint,bbox))
    return kp_list, list_img, crowd

In [12]:
kp_list,list_img,crowd=COCO_Make_dataset(list_json[1])

100%|██████████| 40000/40000 [00:42<00:00, 941.84it/s] 


In [13]:
len(kp_list), len(list_img), len(crowd)

(8331, 8331, 8331)

In [14]:
import numpy as np
kp_list=np.array(kp_list)
list_img=np.array(list_img)
crowd=np.array(crowd)

/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
np.save('/home/lab/MPII_COCO/COCO_Dataset/train2017/list_2_kp',kp_list)
np.save('/home/lab/MPII_COCO/COCO_Dataset/train2017/list_2_img',list_img)
np.save('/home/lab/MPII_COCO/COCO_Dataset/train2017/list_2_crowd',crowd)